In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.matlib

import cv2

from pynq.lib.video import *
from pynq.lib import MicroblazeLibrary

from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

lib = MicroblazeLibrary(base.RPI, ['uart'])
uart_device = lib.uart_open(14,15)# normal uart speed 9600

In [2]:
%%microblaze base.ARDUINO
#include "xio_switch.h"
#include "gpio.h"
#include "timer.h"

#define DEFAULT_PERIOD 625998
#define DEFAULT_DUTY 312998

#define P1A 3
#define P1B 5
#define P2A 6
#define P2B 9
#define P3A 10
#define P3B 11

static timer timer_a;
static timer timer_b;
static timer timer_c;
static timer timer_d;
static timer timer_e;
static timer timer_f;

unsigned int init_ardumoto(){    
    timer_a = timer_open_device(0);
    timer_b = timer_open_device(1);
    timer_c = timer_open_device(2);
    timer_d = timer_open_device(3);
    timer_e = timer_open_device(4);
    timer_f = timer_open_device(5);
    set_pin(3, PWM0);
    set_pin(5, PWM1);
    set_pin(6, PWM2);
    set_pin(9, PWM3);
    set_pin(10, PWM4);
    set_pin(11, PWM5);

    return 0;
}

void run_motor1(unsigned int dir,unsigned int speed){
    if(dir==1){
        timer_pwm_generate(timer_a, DEFAULT_PERIOD, speed*DEFAULT_PERIOD/100);
        timer_pwm_stop(timer_b);
    }
    if(dir==0){
        timer_pwm_generate(timer_b, DEFAULT_PERIOD, speed*DEFAULT_PERIOD/100);
        timer_pwm_stop(timer_a);
    }
}

void stop_motor1()
{
    timer_pwm_stop(timer_a);
    timer_pwm_stop(timer_b);
}

void run_motor2(unsigned int dir,unsigned int speed){
    if(dir==1){
        timer_pwm_generate(timer_c, DEFAULT_PERIOD, speed*DEFAULT_PERIOD/100);
        timer_pwm_stop(timer_d);
    }
    if(dir==0){
        timer_pwm_generate(timer_d, DEFAULT_PERIOD, speed*DEFAULT_PERIOD/100);
        timer_pwm_stop(timer_c);
    }
}

void stop_motor2()
{
    timer_pwm_stop(timer_c);
    timer_pwm_stop(timer_d);
}

void run_motor3(unsigned int dir,unsigned int speed){
    if(dir==1){
        timer_pwm_generate(timer_e, DEFAULT_PERIOD, speed*DEFAULT_PERIOD/100);
        timer_pwm_stop(timer_f);
    }
    if(dir==0){
        timer_pwm_generate(timer_f, DEFAULT_PERIOD, speed*DEFAULT_PERIOD/100);
        timer_pwm_stop(timer_e);
    }
}

void stop_motor3()
{
    timer_pwm_stop(timer_e);
    timer_pwm_stop(timer_f);
}


In [4]:
#####################################################################################

def get_file(file_name,zoom_k):
    point_cloud=[]
    with open(file_name,'r') as file_read:
        print("readin file ", (file_name))
        count=0
        while True:
            line=file_read.readline()
            if not line:
                break
            x_tmp,y_tmp,z_tmp,f_1,f_2,f_3=[float(i) for i in line.split()]
            point = np.zeros((3,1), dtype = np.float) #important for address reflush
            point[0]=x_tmp*zoom_k
            point[1]=y_tmp*zoom_k
            point[2]=z_tmp*zoom_k
            point_cloud.append(point)
            count=count+1
    point_cloud=np.array(point_cloud)
    print(point_cloud)
    print("read finished with ",(len(point_cloud))," points")
    file_read.close()
    return point_cloud

def gen_rotatex(rad_x):
    mat_rotx=np.matlib.identity(4,np.float)
    mat_rotx[1,1]=math.cos(rad_x)
    mat_rotx[1,2]=math.sin(rad_x)
    mat_rotx[2,1]=-1*math.sin(rad_x)
    mat_rotx[2,2]=math.cos(rad_x)
    #print(mat_rotx)
    return mat_rotx

def gen_rotatey(rad_y):
    mat_roty=np.matlib.identity(4,np.float)
    mat_roty[0,0]=math.cos(rad_y)
    mat_roty[0,2]=math.sin(rad_y)
    mat_roty[2,0]=-1*math.sin(rad_y)
    mat_roty[2,2]=math.cos(rad_y)
    #print(mat_roty)
    return mat_roty


def gen_rotatez(rad_z):
    mat_rotz=np.matlib.identity(4,np.float)
    mat_rotz[0,0]=math.cos(rad_z)
    mat_rotz[0,1]=math.sin(rad_z)
    mat_rotz[1,0]=-1*math.sin(rad_z)
    mat_rotz[1,1]=math.cos(rad_z)
    #print(mat_rotz)
    return mat_rotz

def gen_rotmat(rad_x,rad_y,rad_z):
    mat_rotx=gen_rotatex(rad_x)
    mat_roty=gen_rotatey(rad_y)
    mat_rotz=gen_rotatez(rad_z)
    mat_rot=np.matlib.identity(4,np.float)
    mat_rot=mat_rotx*mat_roty*mat_rotz
    return mat_rot

def gen_transmat(x,y,z):
    mat_trans=np.matlib.identity(4,np.float)
    mat_trans[3,0]=x
    mat_trans[3,1]=y
    mat_trans[3,2]=z
    #print(mat_trans)
    return mat_trans

def gen_projection( fov_deg, aspect_retio, near_f, far_f):
    fov_rad = 1.0 / math.tan(fov_deg * 0.5 / 180.0 * math.pi)
    result=np.zeros((4,4))
    result[0,0] = aspect_retio * fov_rad
    result[1,1] = fov_rad
    result[2,2] = far_f / (far_f - near_f)
    result[3,2] = (-far_f * near_f) / (far_f - near_f)
    result[2,3] = 1.0
    result[3,3] = 0.0
    print(result)
    return result

def draw_point(p_x,p_y):
    plt.scatter(p_x,p_y)
    plt.xlabel('x_axis')
    plt.ylabel('y_axis')
    plt.title('test_pcl_draw')
    plt.show()

def draw_point_3d(p_x,p_y,p_z):
    fig=plt.figure();
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(p_x,p_y,p_z,c='k',marker='.',s=0.1)
    plt.show()

def draw_point_cv(image_show):
    pass

def point_simplify(point_input,div_rate):
    ori_size=len(point_input)
    point_output=[]
    count=0
    for point_single in point_input:
        if count==0:
            point_output.append(point_single)
        count=count+1
        if count==div_rate:
            count=0
    return point_output

########################################################################

def in_range(ascii_in):
    if(ascii_in>=48 and ascii_in<=57):
        return 1
    else:
        return 0
    pass


def read_to_float(list_read):
    result=0
    ten_num=2
    for item in list_read:
        result=result+(item-48)*10**(ten_num)
        ten_num=ten_num-1
        pass
    return result
    pass


def get_device_angle(angle_read):
    temp=[0]
    temp1=[]
    read_done=[0,0,0]
    checked=0
    #angle_read=[0,0,0]
    
    while(read_done[0]==0 or read_done[1]==0 or read_done[2]==0 or checked==0):
        
        lib.uart_read(uart_device,temp, 1)
        
        if(temp[0]==97):#selation a
            #print("a")
            count=0
            while(count<3):
                single=[0]
                lib.uart_read(uart_device,single, 1)
                if(temp!=0):
                    temp1.append(single[0])
                    count+=1
                    pass
                pass
            print(temp1)
            angle_1=read_to_float(temp1)
            if(angle_1<500):
                angle_read[0]=angle_1
            else:
                angle_read[0]=angle_1-500
            temp1=[]
            read_done[0]=1
            pass
    
        if(temp[0]==98):#selation b
            #print("b")
            count=0
            while(count<3):
                single=[0]
                lib.uart_read(uart_device,single, 1)
                if(temp!=0):
                    temp1.append(single[0])
                    count+=1
                    pass
                pass
            print(temp1)
            angle_read[1]=read_to_float(temp1)
            temp1=[]
            read_done[1]=1
            pass
    
        if(temp[0]==99):#selation c
            #print("c")
            count=0
            while(count<3):
                single=[0]
                lib.uart_read(uart_device,single, 1)
                if(temp!=0):
                    temp1.append(single[0])
                    count+=1
                    pass
                pass
            print(temp1)
            angle_read[2]=read_to_float(temp1)
            temp1=[]
            read_done[2]=1
            pass     
        
        if(temp[0]==100):#selation d
            #print("c")
            count=0
            check_sum=0
            while(count<3):
                single=[0]
                lib.uart_read(uart_device,single, 1)
                if(temp!=0):
                    temp1.append(single[0])
                    count+=1
                    pass
                pass
            print(temp1)
            check_sum=read_to_float(temp1)
            #print(abs(check_sum-((int((angle_read[0]+angle_read[1]+angle_read[2])/10))+100)))
            if(abs(check_sum-((int((angle_read[0]+angle_read[1]+angle_read[2])/10))+100))<2):#check sum
                checked=1
            temp1=[]
            pass  

    return angle_read

angle_read=[0,0,0]

########################################################################

def forward_kinematics(device_angle,link_len):
    device_end_pos=[0,0,0]
    
    l1=link_len[0]
    l2=link_len[1]
    l3=link_len[2]
    
    a1=device_angle[0]
    a2=device_angle[1]
    a3=device_angle[2]
    
    x_temp=l2*math.cos(a1)*math.cos(a2) + l3*math.cos(a1)*math.cos(a2)*math.cos(a3) - l3*math.cos(a1)*math.sin(a2)*math.sin(a3)
    y_temp=l2*math.cos(a2)*math.sin(a1) + l3*math.cos(a2)*math.cos(a3)*math.sin(a1) - l3*math.sin(a1)*math.sin(a2)*math.sin(a3)
    z_temp=l1 + l2*math.sin(a2) + l3*math.cos(a2)*math.sin(a3) + l3*math.cos(a3)*math.sin(a2)
    
    device_end_pos[0]=x_temp
    device_end_pos[1]=y_temp
    device_end_pos[2]=z_temp
    
    return device_end_pos
    pass

def inverse_dynamatics(device_angle,link_len,end_force):
    device_torque=[0,0,0]
    
    l1=link_len[0]
    l2=link_len[1]
    l3=link_len[2]
    
    a1=device_angle[0]
    a2=device_angle[1]
    a3=device_angle[2]
    
    fx=end_force[0]
    fy=end_force[1]
    fz=end_force[2]
    
    t1_temp=(fy*math.cos(a1))/(l2*math.cos(a1)**2*math.cos(a2) + l2*math.cos(a2)*math.sin(a1)**2 + l3*math.cos(a1)**2*math.cos(a2)*math.cos(a3) + l3*math.cos(a2)*math.cos(a3)*math.sin(a1)**2 - l3*math.cos(a1)**2*math.sin(a2)*math.sin(a3) - l3*math.sin(a1)**2*math.sin(a2)*math.sin(a3)) - (fx*math.sin(a1))/(l2*math.cos(a1)**2*math.cos(a2) + l2*math.cos(a2)*math.sin(a1)**2 + l3*math.cos(a1)**2*math.cos(a2)*math.cos(a3) + l3*math.cos(a2)*math.cos(a3)*math.sin(a1)**2 - l3*math.cos(a1)**2*math.sin(a2)*math.sin(a3) - l3*math.sin(a1)**2*math.sin(a2)*math.sin(a3))
    t2_temp=(fz*(math.cos(a2)*math.sin(a3) + math.cos(a3)*math.sin(a2)))/(l2*math.cos(a2)**2*math.sin(a3) + l2*math.sin(a2)**2*math.sin(a3)) - (fx*math.cos(a1)*(math.sin(a2)*math.sin(a3) - math.cos(a2)*math.cos(a3)))/(l2*math.sin(a1)**2*math.sin(a2)**2*math.sin(a3) + l2*math.cos(a1)**2*math.cos(a2)**2*math.sin(a3) + l2*math.cos(a1)**2*math.sin(a2)**2*math.sin(a3) + l2*math.cos(a2)**2*math.sin(a1)**2*math.sin(a3)) - (fy*math.sin(a1)*(math.sin(a2)*math.sin(a3) - math.cos(a2)*math.cos(a3)))/(l2*math.sin(a1)**2*math.sin(a2)**2*math.sin(a3) + l2*math.cos(a1)**2*math.cos(a2)**2*math.sin(a3) + l2*math.cos(a1)**2*math.sin(a2)**2*math.sin(a3) + l2*math.cos(a2)**2*math.sin(a1)**2*math.sin(a3))
    t3_temp=-1* (fx*(l2*math.cos(a1)*math.cos(a2) + l3*math.cos(a1)*math.cos(a2)*math.cos(a3) - l3*math.cos(a1)*math.sin(a2)*math.sin(a3)))/(l2*l3*math.cos(a1)**2*math.cos(a2)**2*math.sin(a3) + l2*l3*math.cos(a1)**2*math.sin(a2)**2*math.sin(a3) + l2*l3*math.cos(a2)**2*math.sin(a1)**2*math.sin(a3) + l2*l3*math.sin(a1)**2*math.sin(a2)**2*math.sin(a3)) - (fy*(l2*math.cos(a2)*math.sin(a1) + l3*math.cos(a2)*math.cos(a3)*math.sin(a1) - l3*math.sin(a1)*math.sin(a2)*math.sin(a3)))/(l2*l3*math.cos(a1)**2*math.cos(a2)**2*math.sin(a3) + l2*l3*math.cos(a1)**2*math.sin(a2)**2*math.sin(a3) + l2*l3*math.cos(a2)**2*math.sin(a1)**2*math.sin(a3) + l2*l3*math.sin(a1)**2*math.sin(a2)**2*math.sin(a3)) - (fz*(l2*math.sin(a2) + l3*math.cos(a2)*math.sin(a3) + l3*math.cos(a3)*math.sin(a2)))/(l2*l3*math.cos(a2)**2*math.sin(a3) + l2*l3*math.sin(a2)**2*math.sin(a3))
    
    device_torque[0]=t1_temp
    device_torque[1]=t2_temp
    device_torque[2]=t3_temp
    
    return device_torque
    pass

def get_force(end_pos,wall_x,force_k):
    end_force=[0,0,0]
    pos_x=end_pos[0]
    pos_y=end_pos[1]
    pos_z=end_pos[2]
    if(pos_x<wall_x):
        end_force[0]=(wall_x-pos_x)*force_k
        pass
    return end_force


########################################################################

def virtual_env_setup(x_wall_pos):
    env_cloud=[]
    step=16
    axis_po=0
    for i in range(1,2) :
        point_temp_x = np.zeros((3,1), dtype = np.float) #important for address reflush
        point_temp_y = np.zeros((3,1), dtype = np.float)
        point_temp_z = np.zeros((3,1), dtype = np.float)
        point_temp_x[0]=axis_po
        point_temp_y[1]=axis_po
        point_temp_z[2]=axis_po
        env_cloud.append(point_temp_x)
        env_cloud.append(point_temp_y)
        env_cloud.append(point_temp_z)
        axis_po=axis_po+step*2
    
    #for s in range(-3,3):
        #for t in range(-3,3):
            #point_temp_wall = np.zeros((3,1), dtype = np.float)
            #point_temp_wall[0]=x_wall_pos
            #point_temp_wall[1]=t*step*3
            #point_temp_wall[2]=s*step*3
            #env_cloud.append(point_temp_wall)
            #pass
        #pass
    
    env_cloud=np.array(env_cloud)
    return env_cloud
    pass

########################################################################


 
def main():
    #point_cloud=get_file("axis.xyz",3)
    print("point cloud file read done")
    #point_cloud=point_simplify(point_cloud,10)
    x_wall_pos=250
    force_k=0.7
    motor_k=3000
    
    point_cloud=virtual_env_setup(x_wall_pos)
    
    device_angle=[0.2,0,0]
    link_len=[22.5,230,210]
    angle_read=[0,0,0]

    #display setup
    near_f=0.1
    far_f=1000.0
    fov_angle=90.0

    screen_w=640
    screen_h=480

    aspect_ratio=screen_h/screen_w

    display_angle=0
    display_offset={0,0,3.0}   

    mat_projection=gen_projection(fov_angle, aspect_ratio, near_f, far_f)
    
    #hdmi setup
    Mode = VideoMode(640,480,24)
    hdmi_out = base.video.hdmi_out
    hdmi_out.configure(Mode,PIXEL_BGR)
    hdmi_out.start()
    print("HDMI Initialized\n")

    init_ardumoto()#init motorcontrol sub system
    
    for s in range (0,500):
        display_angle+=0.03
        mat_rot=gen_rotmat(-1*math.pi/2,0,display_angle)
        mat_trans=gen_transmat(0,0,3.0)
        
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:]=0
        
        for point_single in point_cloud:
            point_single4=np.zeros((4,1), dtype = np.float)
            point_single4[3]=1.0

            point_single4[0]=point_single[0]
            point_single4[1]=point_single[1]
            point_single4[2]=point_single[2]

            point_rot=mat_rot*point_single4
            point_trans=mat_trans*point_rot
            point_proj=mat_projection*point_trans

            point_proj[0]+=0.5*screen_w
            point_proj[1]+=0.5*screen_h
            #point_proj[0]*=0.5*screen_w
            #point_proj[1]*=0.5*screen_h
            
            y_int=int(point_proj[1])
            x_int=int(point_proj[0])
            if(x_int>640-1-6):
                x_int=640-1-6
            if(x_int<6):
                x_int=6
            if(y_int>480-1-6):
                y_int=480-1-6
            if(y_int<6):
                y_int=6
            
            for t1 in range(-3,3):
                for t2 in range(-3,3):
                    outframe[y_int+t1,x_int+t2,0]=255
                    outframe[y_int+t1,x_int+t2,1]=255

            #output_image[int(point_proj[1]),int(point_proj[0]),2]=255
            
           # outframe[y_int,x_int,0]=0
           # outframe[y_int,x_int,1]=0
            #print("point")
            pass
        
        
        angle_read=get_device_angle(angle_read)
        device_angle_raw=[angle_read[0]-116.1,257.7-angle_read[1],angle_read[2]-290-59]
        print("device angle raw")
        print(device_angle_raw)
        device_rad=[0,0,0]

        device_rad=[math.radians(device_angle_raw[0]),math.radians(device_angle_raw[1]),math.radians(device_angle_raw[2])]
        print(device_angle_raw)
        
        end_pos=forward_kinematics(device_rad,link_len)
        end_force=get_force(end_pos,x_wall_pos,force_k)
        #print("force")
        #print(end_force)
        
        device_torque=inverse_dynamatics(device_rad,link_len,end_force)
        #print("torque")
        #print(device_torque)
        
        motor_torque=[0,0,0]
        motor_torque[0]=motor_k*device_torque[0]
        motor_torque[1]=motor_k*device_torque[1]
        motor_torque[2]=motor_k*device_torque[2]
        #print("sent torque")
        #print(motor_torque)
        
        if(motor_torque[0]!=0 and motor_torque[1]!=0 and motor_torque[2]!=0):
            s_t1=int(motor_torque[0])
            s_t2=int(motor_torque[1])
            s_t3=int(motor_torque[2])
            
            if(s_t1>30):
                s_t1=30
            if(s_t1<-30):
                s_t1=-30
            
            if(s_t2>30):
                s_t2=30
            if(s_t2<-30):
                s_t2=-30
                
            if(s_t3>30):
                s_t3=30
            if(s_t3<-30):
                s_t3=-30
                
            if(s_t1>0):
                run_motor1(0,s_t1)
                pass
            if(s_t1<0):
                run_motor1(1,-1*s_t1)
            
            if(s_t2>0):
                run_motor2(0,s_t2)
                pass
            if(s_t2<0):
                run_motor2(1,-1*s_t2)
                
            if(s_t3>0):
                run_motor3(0,s_t3)
                pass
            if(s_t3<0):
                run_motor3(1,-1*s_t3)

                pass
            pass
        else:
            stop_motor1()
            stop_motor2()
            stop_motor3()
            pass
        
        end_pos4=np.zeros((4,1), dtype = np.float)
        for i in range(0,3):
            end_pos4[i]=end_pos[i]
        end_pos4[3]=1.0
        
        pos_rot=mat_rot*end_pos4
        pos_trans=mat_trans*pos_rot
        pos_proj=mat_projection*pos_trans
        
        pos_proj[0]+=0.5*screen_w
        pos_proj[1]+=0.5*screen_h
        
        y_int=int(pos_proj[1])
        x_int=int(pos_proj[0])
        if(x_int>640-1-12):
            x_int=640-1-12
        if(x_int<12):
            x_int=12
        if(y_int>480-1-12):
            y_int=480-1-12
        if(y_int<12):
            y_int=12
        
        for t1 in range(-6,6):
            for t2 in range(-6,6):
                outframe[y_int+t1,x_int+t2,0]=255
                outframe[y_int+t1,x_int+t2,2]=255
        
        cv2.rectangle(outframe,(x_int,y_int),(x_int+3,y_int+3),(255,0,0),2)
        print(y_int,x_int,"pos")

        print("cycle test")
        hdmi_out.writeframe(outframe)
        print("frame output finished")
        
    pass
    hdmi_out.stop()
    del hdmi_out

 
if __name__ == '__main__':
    main()
    print ('now __name__ is %s' %__name__)


point cloud file read done
[[ 0.75        0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.00010001  1.        ]
 [ 0.          0.         -0.10001     0.        ]]
HDMI Initialized



KeyboardInterrupt: 